# PointNets


Disclaimer: The visualization files (render_balls_so.cpp, render_balls_so.dll, show3d_balls.py, Show3dBalls.ipynb) and the dataloader (datasets.py) are from https://github.com/fxia22/pointnet.pytorch.

In [1]:
from __future__ import print_function
from show3d_balls import *
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pdb
import torch.nn.functional as F
import numpy as np
import torchvision.datasets as dset
from datasets import PartDataset



### Networks - Benchmark

In [2]:
class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2,1)
        x = torch.bmm(x, trans)
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans

class PointNetCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetCls, self).__init__()
        self.feat = PointNetfeat(global_feat=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop = nn.Dropout2d(0.3)
        self.relu = nn.ReLU()
    def forward(self, x):
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=0), trans

class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feat = PointNetfeat(global_feat=False)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans



### Networks - Experimentation

In [ ]:
class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2,1)
        x = torch.bmm(x, trans)
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans

class PointNetCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetCls, self).__init__()
        self.feat = PointNetfeat(global_feat=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop = nn.Dropout2d(0.7)
        self.relu = nn.ReLU()
    def forward(self, x):
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
#         x = self.drop(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=0), trans

class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feat = PointNetfeat(global_feat=False)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(64, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 64)
        self.drop = nn.Dropout2d(0.3)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.drop(x)
        x = self.fc2(x)
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans



### Classification Data Loader

In [3]:


parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=16, help='input batch size')
parser.add_argument('--num_points', type=int, default=2500, help='input batch size')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
# real default 25
# parser.add_argument('--nepoch', type=int, default=3, help='number of epochs to train for')
parser.add_argument('--nepoch', type=int, default=6, help='number of epochs to train for')
parser.add_argument('--outf', type=str, default='cls',  help='output folder')
parser.add_argument('--model', type=str, default = '',  help='model path')

opt, unknown = parser.parse_known_args()
print (opt)

blue = lambda x:'\033[94m' + x + '\033[0m'

opt.manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = True, npoints = opt.num_points)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = True, train = False, npoints = opt.num_points)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

print(len(dataset), len(test_dataset))
num_classes = len(dataset.classes)
print('classes', num_classes)

try:
    os.makedirs(opt.outf)
except OSError:
    pass


Namespace(batchSize=16, model='', nepoch=6, num_points=2500, outf='cls', workers=4)
Random Seed:  6055
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
15990 1785
classes 16


### Classification Training

In [4]:
# # classifier = PointNetCls(k = num_classes, num_points = opt.num_points)
classifier = PointNetCls(k = num_classes)


if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))


optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
classifier.cuda()

num_batch = len(dataset)/opt.batchSize
train_accuracy = []
train_loss = []
test_accuracy = []
test_loss = []
print_to_file = True
for epoch in range(opt.nepoch):
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points, target = Variable(points), Variable(target[:,0])
        points = points.transpose(2,1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        loss = F.nll_loss(pred, target)
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' %(epoch, i, num_batch, loss.item(),correct.item() / float(opt.batchSize)))
        if print_to_file == True:
            train_accuracy.append(correct.item() / float(opt.batchSize))
            train_loss.append(loss.item())

#         try i % 10 == 9
        if i % 10 == 9:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points, target = Variable(points), Variable(target[:,0])
            points = points.transpose(2,1)
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _ = classifier(points)
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' %(epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batchSize)))
            if print_to_file == True:
                test_accuracy.append(correct.item() / float(opt.batchSize))
                test_loss.append(loss.item())

    torch.save(classifier.state_dict(), '%s/cls_model_%d.pth' % (opt.outf, epoch))
    np.savetxt('%s/cls_train_loss_%d.out' % (opt.outf, epoch), train_loss)
    np.savetxt('%s/cls_train_acc_%d.out' % (opt.outf, epoch), train_accuracy)
    np.savetxt('%s/cls_test_loss_%d.out' % (opt.outf, epoch), test_loss)
    np.savetxt('%s/cls_test_acc_%d.out' % (opt.outf, epoch), test_accuracy)



[0: 0/999] train loss: 2.740195 accuracy: 0.062500
[0: 1/999] train loss: 2.467044 accuracy: 0.312500
[0: 2/999] train loss: 2.423957 accuracy: 0.687500
[0: 3/999] train loss: 2.426685 accuracy: 0.375000
[0: 4/999] train loss: 2.170271 accuracy: 0.625000
[0: 5/999] train loss: 2.171161 accuracy: 0.562500
[0: 6/999] train loss: 2.156839 accuracy: 0.750000
[0: 7/999] train loss: 2.035604 accuracy: 0.750000
[0: 8/999] train loss: 1.959406 accuracy: 0.687500
[0: 9/999] train loss: 2.075765 accuracy: 0.625000
[0: 9/999] test loss: 2.770398 accuracy: 0.250000
[0: 10/999] train loss: 2.010327 accuracy: 0.687500
[0: 11/999] train loss: 2.119379 accuracy: 0.750000
[0: 12/999] train loss: 1.678393 accuracy: 0.812500
[0: 13/999] train loss: 1.923393 accuracy: 0.562500
[0: 14/999] train loss: 1.784420 accuracy: 0.875000
[0: 15/999] train loss: 1.912089 accuracy: 0.812500
[0: 16/999] train loss: 1.803008 accuracy: 0.750000
[0: 17/999] train loss: 1.520988 accuracy: 0.937500
[0: 18/999] train loss: 

[0: 143/999] train loss: 1.313040 accuracy: 0.875000
[0: 144/999] train loss: 1.217062 accuracy: 0.937500
[0: 145/999] train loss: 1.823278 accuracy: 0.625000
[0: 146/999] train loss: 1.275205 accuracy: 0.937500
[0: 147/999] train loss: 1.264833 accuracy: 0.812500
[0: 148/999] train loss: 1.414168 accuracy: 0.937500
[0: 149/999] train loss: 1.447090 accuracy: 0.750000
[0: 149/999] test loss: 1.426595 accuracy: 0.875000
[0: 150/999] train loss: 1.391788 accuracy: 0.687500
[0: 151/999] train loss: 1.282696 accuracy: 0.750000
[0: 152/999] train loss: 1.423163 accuracy: 0.687500
[0: 153/999] train loss: 1.481425 accuracy: 0.625000
[0: 154/999] train loss: 1.396206 accuracy: 0.812500
[0: 155/999] train loss: 1.547379 accuracy: 0.812500
[0: 156/999] train loss: 1.553902 accuracy: 0.750000
[0: 157/999] train loss: 1.608374 accuracy: 0.687500
[0: 158/999] train loss: 1.206018 accuracy: 0.750000
[0: 159/999] train loss: 1.069633 accuracy: 0.875000
[0: 159/999] test loss: 1.371687 accuracy: 0.81

[0: 283/999] train loss: 1.190406 accuracy: 0.937500
[0: 284/999] train loss: 1.011722 accuracy: 0.812500
[0: 285/999] train loss: 1.216876 accuracy: 0.812500
[0: 286/999] train loss: 1.285818 accuracy: 0.750000
[0: 287/999] train loss: 1.385902 accuracy: 0.625000
[0: 288/999] train loss: 1.577864 accuracy: 0.812500
[0: 289/999] train loss: 1.552973 accuracy: 0.750000
[0: 289/999] test loss: 1.113414 accuracy: 0.875000
[0: 290/999] train loss: 1.101632 accuracy: 0.937500
[0: 291/999] train loss: 1.676424 accuracy: 0.625000
[0: 292/999] train loss: 1.675808 accuracy: 0.875000
[0: 293/999] train loss: 1.055436 accuracy: 0.812500
[0: 294/999] train loss: 1.253690 accuracy: 0.750000
[0: 295/999] train loss: 1.365862 accuracy: 0.750000
[0: 296/999] train loss: 1.179024 accuracy: 0.812500
[0: 297/999] train loss: 1.096346 accuracy: 0.937500
[0: 298/999] train loss: 1.509043 accuracy: 0.562500
[0: 299/999] train loss: 1.434025 accuracy: 0.812500
[0: 299/999] test loss: 1.164384 accuracy: 0.81

[0: 423/999] train loss: 1.161748 accuracy: 0.937500
[0: 424/999] train loss: 1.130144 accuracy: 0.812500
[0: 425/999] train loss: 1.427282 accuracy: 0.812500
[0: 426/999] train loss: 1.540770 accuracy: 0.625000
[0: 427/999] train loss: 1.271165 accuracy: 0.812500
[0: 428/999] train loss: 1.329602 accuracy: 0.750000
[0: 429/999] train loss: 1.353521 accuracy: 0.875000
[0: 429/999] test loss: 1.249573 accuracy: 1.000000
[0: 430/999] train loss: 1.523684 accuracy: 0.812500
[0: 431/999] train loss: 1.370676 accuracy: 0.687500
[0: 432/999] train loss: 1.365493 accuracy: 0.750000
[0: 433/999] train loss: 1.103462 accuracy: 0.812500
[0: 434/999] train loss: 1.041286 accuracy: 0.812500
[0: 435/999] train loss: 0.934238 accuracy: 0.937500
[0: 436/999] train loss: 1.325987 accuracy: 0.750000
[0: 437/999] train loss: 1.325446 accuracy: 0.812500
[0: 438/999] train loss: 1.217848 accuracy: 0.875000
[0: 439/999] train loss: 1.866432 accuracy: 0.562500
[0: 439/999] test loss: 1.135377 accuracy: 0.87

[0: 563/999] train loss: 1.125308 accuracy: 0.812500
[0: 564/999] train loss: 1.243259 accuracy: 0.750000
[0: 565/999] train loss: 1.176244 accuracy: 0.812500
[0: 566/999] train loss: 1.180693 accuracy: 0.687500
[0: 567/999] train loss: 1.545692 accuracy: 0.687500
[0: 568/999] train loss: 1.259668 accuracy: 0.750000
[0: 569/999] train loss: 1.677592 accuracy: 0.562500
[0: 569/999] test loss: 1.188744 accuracy: 0.937500
[0: 570/999] train loss: 1.249835 accuracy: 0.875000
[0: 571/999] train loss: 1.215811 accuracy: 0.750000
[0: 572/999] train loss: 1.329992 accuracy: 0.750000
[0: 573/999] train loss: 1.353535 accuracy: 0.750000
[0: 574/999] train loss: 1.246285 accuracy: 0.750000
[0: 575/999] train loss: 1.572937 accuracy: 0.562500
[0: 576/999] train loss: 1.170289 accuracy: 0.875000
[0: 577/999] train loss: 1.441279 accuracy: 0.875000
[0: 578/999] train loss: 0.928247 accuracy: 0.937500
[0: 579/999] train loss: 1.046013 accuracy: 0.937500
[0: 579/999] test loss: 1.135743 accuracy: 0.75

...

[5: 346/999] train loss: 1.402797 accuracy: 0.750000
[5: 347/999] train loss: 0.953591 accuracy: 1.000000
[5: 348/999] train loss: 1.473470 accuracy: 0.687500
[5: 349/999] train loss: 1.167054 accuracy: 0.812500
[5: 349/999] test loss: 1.034940 accuracy: 0.937500
[5: 350/999] train loss: 1.396810 accuracy: 0.562500
[5: 351/999] train loss: 1.214204 accuracy: 0.750000
[5: 352/999] train loss: 1.339904 accuracy: 0.687500
[5: 353/999] train loss: 1.069054 accuracy: 0.812500
[5: 354/999] train loss: 1.039223 accuracy: 0.812500
[5: 355/999] train loss: 0.992888 accuracy: 0.875000
[5: 356/999] train loss: 1.316344 accuracy: 0.687500
[5: 357/999] train loss: 1.627995 accuracy: 0.687500
[5: 358/999] train loss: 1.473864 accuracy: 0.812500
[5: 359/999] train loss: 1.157227 accuracy: 0.750000
[5: 359/999] test loss: 1.440024 accuracy: 0.625000
[5: 360/999] train loss: 1.278082 accuracy: 0.687500
[5: 361/999] train loss: 1.563610 accuracy: 0.500000
[5: 362/999] train loss: 1.098338 accuracy: 0.81

[5: 486/999] train loss: 1.390786 accuracy: 0.687500
[5: 487/999] train loss: 1.261650 accuracy: 0.750000
[5: 488/999] train loss: 1.529373 accuracy: 0.562500
[5: 489/999] train loss: 1.776326 accuracy: 0.437500
[5: 489/999] test loss: 1.092357 accuracy: 0.812500
[5: 490/999] train loss: 1.180137 accuracy: 0.750000
[5: 491/999] train loss: 0.872047 accuracy: 0.812500
[5: 492/999] train loss: 1.137381 accuracy: 0.687500
[5: 493/999] train loss: 1.149105 accuracy: 0.687500
[5: 494/999] train loss: 1.356133 accuracy: 0.750000
[5: 495/999] train loss: 1.120232 accuracy: 0.750000
[5: 496/999] train loss: 1.202307 accuracy: 0.812500
[5: 497/999] train loss: 1.116804 accuracy: 0.750000
[5: 498/999] train loss: 1.623433 accuracy: 0.437500
[5: 499/999] train loss: 0.970572 accuracy: 0.812500
[5: 499/999] test loss: 1.022409 accuracy: 0.875000
[5: 500/999] train loss: 1.130017 accuracy: 0.875000
[5: 501/999] train loss: 1.196405 accuracy: 0.625000
[5: 502/999] train loss: 1.121384 accuracy: 0.75

[5: 626/999] train loss: 1.224138 accuracy: 0.687500
[5: 627/999] train loss: 1.194047 accuracy: 0.687500
[5: 628/999] train loss: 1.090594 accuracy: 0.875000
[5: 629/999] train loss: 1.104723 accuracy: 0.812500
[5: 629/999] test loss: 1.293878 accuracy: 0.687500
[5: 630/999] train loss: 1.319358 accuracy: 0.812500
[5: 631/999] train loss: 1.186876 accuracy: 0.812500
[5: 632/999] train loss: 1.264598 accuracy: 0.687500
[5: 633/999] train loss: 1.052690 accuracy: 0.875000
[5: 634/999] train loss: 1.179492 accuracy: 0.875000
[5: 635/999] train loss: 1.529180 accuracy: 0.500000
[5: 636/999] train loss: 1.186386 accuracy: 0.687500
[5: 637/999] train loss: 0.922976 accuracy: 0.812500
[5: 638/999] train loss: 0.884566 accuracy: 0.937500
[5: 639/999] train loss: 1.313893 accuracy: 0.687500
[5: 639/999] test loss: 1.024553 accuracy: 0.875000
[5: 640/999] train loss: 0.976513 accuracy: 0.750000
[5: 641/999] train loss: 0.903920 accuracy: 0.812500
[5: 642/999] train loss: 1.225640 accuracy: 0.81

[5: 766/999] train loss: 1.205975 accuracy: 0.625000
[5: 767/999] train loss: 1.479688 accuracy: 0.500000
[5: 768/999] train loss: 1.198470 accuracy: 0.750000
[5: 769/999] train loss: 0.958134 accuracy: 0.625000
[5: 769/999] test loss: 0.876645 accuracy: 0.937500
[5: 770/999] train loss: 0.988745 accuracy: 0.875000
[5: 771/999] train loss: 1.094076 accuracy: 0.687500
[5: 772/999] train loss: 1.367780 accuracy: 0.812500
[5: 773/999] train loss: 1.096515 accuracy: 0.750000
[5: 774/999] train loss: 1.445583 accuracy: 0.812500
[5: 775/999] train loss: 1.058008 accuracy: 0.875000
[5: 776/999] train loss: 0.997696 accuracy: 0.812500
[5: 777/999] train loss: 0.726509 accuracy: 0.937500
[5: 778/999] train loss: 1.116943 accuracy: 0.812500
[5: 779/999] train loss: 1.567157 accuracy: 0.500000
[5: 779/999] test loss: 1.591772 accuracy: 0.750000
[5: 780/999] train loss: 1.033184 accuracy: 0.687500
[5: 781/999] train loss: 1.451035 accuracy: 0.500000
[5: 782/999] train loss: 0.778557 accuracy: 0.81

[5: 906/999] train loss: 1.275381 accuracy: 0.562500
[5: 907/999] train loss: 1.060685 accuracy: 0.687500
[5: 908/999] train loss: 1.085396 accuracy: 0.812500
[5: 909/999] train loss: 1.067493 accuracy: 0.750000
[5: 909/999] test loss: 1.228295 accuracy: 0.812500
[5: 910/999] train loss: 1.444134 accuracy: 0.687500
[5: 911/999] train loss: 1.648361 accuracy: 0.750000
[5: 912/999] train loss: 1.250498 accuracy: 0.812500
[5: 913/999] train loss: 1.133451 accuracy: 0.937500
[5: 914/999] train loss: 1.479217 accuracy: 0.562500
[5: 915/999] train loss: 0.810171 accuracy: 0.937500
[5: 916/999] train loss: 0.990318 accuracy: 0.750000
[5: 917/999] train loss: 0.971675 accuracy: 0.875000
[5: 918/999] train loss: 1.247457 accuracy: 0.750000
[5: 919/999] train loss: 1.669285 accuracy: 0.562500
[5: 919/999] test loss: 0.907919 accuracy: 0.875000
[5: 920/999] train loss: 1.122117 accuracy: 0.812500
[5: 921/999] train loss: 0.790353 accuracy: 0.812500
[5: 922/999] train loss: 0.856485 accuracy: 0.81

### Classification Display

In [ ]:


#showpoints(np.random.randn(2500,3), c1 = np.random.uniform(0,1,size = (2500)))

parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, default = 'cls/cls_model_2.pth',  help='model path')
parser.add_argument('--num_points', type=int, default=2500, help='input batch size')


opt, unknown = parser.parse_known_args()
print (opt)

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0' , train = False, classification = True,  npoints = opt.num_points)

testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle = True)


# classifier = PointNetCls(k = len(test_dataset.classes), num_points = opt.num_points)
classifier = PointNetCls(k = len(test_dataset.classes))
classifier.cuda()
classifier.load_state_dict(torch.load(opt.model))
classifier.eval()


for i, data in enumerate(testdataloader, 0):
    points, target = data
    points, target = Variable(points), Variable(target[:, 0])
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    pred, _ = classifier(points)
    loss = F.nll_loss(pred, target)

    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    print('i:%d  loss: %f accuracy: %f' %(i, loss.data[0], correct/float(32)))


### Segmentation Data Loader

In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=32, help='input batch size')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
# real default 25
parser.add_argument('--nepoch', type=int, default=3, help='number of epochs to train for')
parser.add_argument('--outf', type=str, default='seg',  help='output folder')
parser.add_argument('--model', type=str, default = '',  help='model path')

opt, unknown = parser.parse_known_args()
print (opt)

opt.manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = False, class_choice = ['Chair'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

test_dataset = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', classification = False, class_choice = ['Chair'], train = False)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batchSize, drop_last=True,
                                          shuffle=True, num_workers=int(opt.workers))

print(len(dataset), len(test_dataset))
num_classes = dataset.num_seg_classes
print('classes', num_classes)
try:
    os.makedirs(opt.outf)
except OSError:
    pass

blue = lambda x:'\033[94m' + x + '\033[0m'


### Segmentation Training

In [ ]:
classifier = PointNetDenseCls(k = num_classes)

if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))

optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
classifier.cuda()

num_batch = len(dataset)/opt.batchSize

for epoch in range(opt.nepoch):
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points, target = Variable(points), Variable(target)
        points = points.transpose(2,1) 
        points, target = points.cuda(), target.cuda()   
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1,1)[:,0] - 1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' %(epoch, i, num_batch, loss.item(), correct.item()/float(opt.batchSize * 2500)))
        
        if i % 10 == 0:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points, target = Variable(points), Variable(target)
            points = points.transpose(2,1) 
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1,1)[:,0] - 1

            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' %(epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batchSize * 2500)))
    
    torch.save(classifier.state_dict(), '%s/seg_model_%d.pth' % (opt.outf, epoch))

### Segmentation Display

In [ ]:

#showpoints(np.random.randn(2500,3), c1 = np.random.uniform(0,1,size = (2500)))

parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, default = 'seg/seg_model_2.pth',  help='model path')
parser.add_argument('--idx', type=int, default = 0,   help='model index')



opt, unknown = parser.parse_known_args()
print (opt)

d = PartDataset(root = '../../shapenetcore_partanno_segmentation_benchmark_v0', class_choice = ['Lamp'], train = False)

idx = opt.idx

print("model %d/%d" %( idx, len(d)))

point, seg = d[idx]
print(point.size(), seg.size())

point_np = point.numpy()



cmap = plt.cm.get_cmap("hsv", 10)
cmap = np.array([cmap(i) for i in range(10)])[:,:3]
gt = cmap[seg.numpy() - 1, :]

classifier = PointNetDenseCls(k = 4)
classifier.load_state_dict(torch.load(opt.model))
classifier.eval()

point = point.transpose(1,0).contiguous()

point = Variable(point.view(1, point.size()[0], point.size()[1]))
pred, _ = classifier(point)
pred_choice = pred.data.max(2)[1]
print(pred_choice)

#print(pred_choice.size())
pred_color = cmap[pred_choice.numpy()[0], :]

#print(pred_color.shape)
showpoints(point_np, gt, pred_color)
